# ML-assignment 3

## Part 0
* Hospital/bank/customer customer service - how many males/females are calling
* 

### Streaming
* Yes, with the proper data preprocessing it would be possible. We only need to extract the chosen features. We would probably have to extract only the last small part (maybe 1 second) the data in order to identify the gender of the person currently talking. However this would probably only work with one person talking at a time.